<a href="https://colab.research.google.com/github/amrutadeo-22/resnet/blob/main/IdentityMapping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
# hello
class Bottleneck(nn.Module):
    def __init__(self, in_planes, out_planes, stride=1):
        super(Bottleneck, self).__init__()
        bottleneck_planes = out_planes // 4
        self.bn1 = nn.BatchNorm2d(in_planes)
        self.conv1 = nn.Conv2d(in_planes, bottleneck_planes, kernel_size=1, bias=False)
        self.bn2 = nn.BatchNorm2d(bottleneck_planes)
        self.conv2 = nn.Conv2d(bottleneck_planes, bottleneck_planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn3 = nn.BatchNorm2d(bottleneck_planes)
        self.conv3 = nn.Conv2d(bottleneck_planes, out_planes, kernel_size=1, bias=False)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != out_planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(x))
        shortcut = self.shortcut(out)
        out = F.relu(self.bn2(self.conv1(out)))
        out = F.relu(self.bn3(self.conv2(out)))
        out = self.conv3(out)
        out += shortcut
        return out

class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 16

        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1, bias=False)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.bn = nn.BatchNorm2d(256)
        self.linear = nn.Linear(256, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.conv1(x))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = F.relu(self.bn(out))
        out = F.avg_pool2d(out, 8)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out

def ResNet164():
    return ResNet(Bottleneck, [18, 18, 18])

# Check if CUDA is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Example usage:
model = ResNet164().to(device)

# To create a random input tensor
input_tensor = torch.randn(1, 3, 32, 32).to(device)

# Forward pass
output = model(input_tensor)
print(output)


tensor([[ 0.0805,  0.2851,  0.1384,  0.0115,  0.0298, -0.2005, -0.2607,  0.2191,
          0.0756, -0.4211]], grad_fn=<AddmmBackward0>)


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

# Define the ResNet model with Bottleneck blocks
class Bottleneck(nn.Module):
    def __init__(self, in_planes, out_planes, stride=1):
        super(Bottleneck, self).__init__()
        bottleneck_planes = out_planes // 4
        self.bn1 = nn.BatchNorm2d(in_planes)
        self.conv1 = nn.Conv2d(in_planes, bottleneck_planes, kernel_size=1, bias=False)
        self.bn2 = nn.BatchNorm2d(bottleneck_planes)
        self.conv2 = nn.Conv2d(bottleneck_planes, bottleneck_planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn3 = nn.BatchNorm2d(bottleneck_planes)
        self.conv3 = nn.Conv2d(bottleneck_planes, out_planes, kernel_size=1, bias=False)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != out_planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(x))
        shortcut = self.shortcut(out)
        out = F.relu(self.bn2(self.conv1(out)))
        out = F.relu(self.bn3(self.conv2(out)))
        out = self.conv3(out)
        out += shortcut
        return out

class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 16

        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1, bias=False)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.bn = nn.BatchNorm2d(256)
        self.linear = nn.Linear(256, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.conv1(x))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = F.relu(self.bn(out))
        out = F.avg_pool2d(out, 8)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out

def ResNet164():
    return ResNet(Bottleneck, [18, 18, 18])

# Check if CUDA is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Data transformations
transform = transforms.Compose(
    [transforms.RandomCrop(32, padding=4),
     transforms.RandomHorizontalFlip(),
     transforms.ToTensor(),
     transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])

# Load CIFAR-10 dataset
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = DataLoader(trainset, batch_size=128, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = DataLoader(testset, batch_size=100, shuffle=False, num_workers=2)

# Define the model, loss function, and optimizer
model = ResNet164().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training the model
def train(epoch):
    model.train()
    running_loss = 0.0
    for i, (inputs, targets) in enumerate(trainloader):
        inputs, targets = inputs.to(device), targets.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 100 == 99:
            print(f'Epoch {epoch+1}, Batch {i+1}, Loss: {running_loss / 100:.3f}')
            running_loss = 0.0

# Testing the model
def test():
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, targets in testloader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += targets.size(0)
            correct += (predicted == targets).sum().item()

    print(f'Accuracy of the network on the 10,000 test images: {100 * correct / total:.2f}%')


for epoch in range(10):
    train(epoch)
    test()


Files already downloaded and verified
Files already downloaded and verified
Epoch 1, Batch 100, Loss: 2.134
Epoch 1, Batch 200, Loss: 1.817
Epoch 1, Batch 300, Loss: 1.673
Accuracy of the network on the 10,000 test images: 41.16%
Epoch 2, Batch 100, Loss: 1.436
Epoch 2, Batch 200, Loss: 1.347
Epoch 2, Batch 300, Loss: 1.291
Accuracy of the network on the 10,000 test images: 55.51%
Epoch 3, Batch 100, Loss: 1.152
Epoch 3, Batch 200, Loss: 1.095
Epoch 3, Batch 300, Loss: 1.053
Accuracy of the network on the 10,000 test images: 63.55%
Epoch 4, Batch 100, Loss: 0.943
Epoch 4, Batch 200, Loss: 0.946
Epoch 4, Batch 300, Loss: 0.925
Accuracy of the network on the 10,000 test images: 68.32%
Epoch 5, Batch 100, Loss: 0.835
Epoch 5, Batch 200, Loss: 0.835
Epoch 5, Batch 300, Loss: 0.804
Accuracy of the network on the 10,000 test images: 71.92%
Epoch 6, Batch 100, Loss: 0.741
Epoch 6, Batch 200, Loss: 0.715
Epoch 6, Batch 300, Loss: 0.704
Accuracy of the network on the 10,000 test images: 75.66%
